In [1]:
%load_ext autoreload

%autoreload 2
import random

import matplotlib.pyplot as plt
import numpy as np

from data_loader import data_extractor
from hcope import Hcope
from mdp.environments import cartpole as cp
from mdp.history import History
from mdp.policies.f_policy import FBSoftmax
from policy_evaluation import tabp_carp_evaluation
import hcope

MODE = 1  # 0 for self generated; 1 for submission
goodtheta22 = [
    7099.66,
    2271.2,
    -2649.36,
    -792.266,
    -6800.38,
    -2501.96,
    -6709.64,
    -1423.85,
    6271.33,
    1026.15,
    -5668.35,
    -4211.29,
    -4798.36,
    -2831.35,
    4244.49,
    828.706,
    6024.58,
    4662.6,
    -2009.16,
    -5350.25,
    1622.12,
    -2141.27,
    2725.41,
    6408.9,
    -1421.81,
    2898.36,
    -2770.57,
    -97.896,
    -4250.56,
    26.9385,
    -6994.73,
    -1322.33,
    3483.53,
    1609.17,
    7254.78,
    1908.97,
    -7299.73,
    -3617.97,
    3351.57,
    -658.239,
    6817.05,
    3871.37,
    5924.42,
    2584.16,
    -5977.88,
    -4781.25,
    4289.53,
    7169.24,
    2758.42,
    5702.44,
    -3217.19,
    2276.16,
    -4198.01,
    -7123.93,
    -5158.91,
    174.284,
    211.946,
    1144.41,
    4225.33,
    -1377.48,
    6675.24,
    169.09,
    -4384.77,
    -2024,
    5585.64,
    1410.12,
    7342.91,
    2935.01,
    -4698.38,
    331.438,
    -7620.29,
    -3431.11,
    4557.3,
    7181.87,
    -2331.42,
    4737.3,
    -4677.39,
    -7077.93,
    -2921.54,
    -5814.33,
    4416.95,
    13567.5,
    1443.74,
    -9497.78,
    -951.515,
    -13231.5,
    -1710.83,
    -13593.6,
    -453.448,
    11952.8,
    560.045,
    -11977.6,
    -2807.75,
    -11460.6,
    -1726.2,
    10513.8,
    1071.06,
    12521.1,
    2831.24,
    -8600.44,
    -2743.97,
    8099.74,
    701.891,
    9484.37,
    4070.9,
    6186.7,
    1071.19,
    -9126.72,
    824.822,
    -10862,
    511.135,
    -14032.8,
    -866.641,
    7830.5,
    1238.87,
    12184,
    219.301,
    -14601.3,
    -3119.78,
    8216.25,
    -365.459,
    12222.1,
    1773.97,
    12605.2,
    1249.98,
    -9479.31,
    -466.794,
    11349.3,
    4226.63,
    10468.9,
    2441.88,
    -8230.38,
    -977.918,
    -10209.5,
    -3087.42,
    -16359.3,
    -2584.55,
    5504.88,
    -1368.39,
    11182,
    -1292.38,
    15323.6,
    723.609,
    -7116.13,
    -3406.06,
    13391.4,
    2323.59,
    16696.1,
    3629.58,
    -8394.22,
    899.368,
    -13126.9,
    -1291.3,
    15538.5,
    5617.21,
    -8574.09,
    1649.43,
    -12740.2,
    -4336.8,
    -12669.2,
    -2908.52,
    8915.04
]

goodtheta22 = [2,-2,2,-2,2,-2,2,-2,2,-2]
goodtheta22 = np.array(goodtheta22)

import inspect
import re
def slog(x):
    """log var status with name"""
    frame = inspect.currentframe().f_back
    s = inspect.getframeinfo(frame).code_context[0]
    r = re.search(r"\((.*)\)", s).group(1)
    print("{} = {}".format(r,x))

In [2]:
MODE = 0
if MODE == 1:
    num_state_features, num_actions, k, theta_b, num_episodes, states, actions, rewards, pi_b_St_At = data_extractor("../data/test_data/data.csv")
    print(num_state_features, num_actions, k, theta_b, num_episodes)
    D = []
    
    num_episodes = 1000
    for i in range(num_episodes):
        history = History(states[i], actions[i], rewards[i])
        D.append(history)
    
    J_pi_b_hat = 0.0
    for H in D:
        J_pi_b_hat += np.sum(H.rewards)
    
    J_pi_b_hat /= len(D)
    
    policy = FBSoftmax(num_state_features,num_actions,k,0)
    policy.parameters = theta_b
    hc = Hcope(D, policy, int(len(D)*0.6), 2)
    for i in range(100):
#         print("step",i)
        theta = hc.sample_theta_c()
    
        slog(theta)

        test_policy = FBSoftmax(num_state_features,num_actions,k,0)
        test_policy.parameters = theta
        test_pdis = hc.test_pdis(test_policy)
        slog(test_pdis)
#     hc.plot_es()
    
elif MODE == 0:
    iO = 1
    dO = 0
    policy = FBSoftmax(4,2,iO,dO)
    history_size = 200

    environment = cp.Cartpole

    t = tabp_carp_evaluation(policy, environment)
    
    slog(policy.parameters.shape)

    mean_J_hat, D = t.run_policy(goodtheta22, history_size)

    slog(mean_J_hat)

     
    hc = Hcope(D, policy, int(len(D)*0.6), mean_J_hat)
    for i in range(100):
        print("step",i)
        theta = hc.sample_theta_c()
        policy = FBSoftmax(4,2,iO,dO)
        numEpisode = 100

        environment = cp.Cartpole

        t = tabp_carp_evaluation(policy, environment)

        mean_J_hat_after, D = t.run_policy(theta.reshape(policy.parameters.shape), numEpisode)
        slog(theta)
        slog(mean_J_hat_after)
        
        test_policy = FBSoftmax(4,2,iO,dO)
        test_policy.parameters = theta
        test_pdis = hc.test_pdis(test_policy)
        slog(test_pdis)

    

policy.parameters.shape = (10,)
mean_J_hat = 28.035
(5_w,10)-aCMA-ES (mu_w=3.2,w_1=45%) in dimension 10 (seed=193017, Wed Dec 11 20:06:57 2019)
step 0
getting new theta
[999967.6765980028, 999973.0599810529, 999963.1202491157, 999966.497680729, 999961.9373464335, 999972.8322437753, 999979.2911404893, 999972.0189258102, 999972.6411253829, 999976.6417373752]
told
theta = [ 2.14747488 -1.22065843  2.05879336 -2.11590456  2.207024   -1.87395998
  1.26267058 -2.59934248  1.67832205 -2.05517522]
mean_J_hat_after = 23.56
test_pdis = 20.189980075371246
step 1
getting new theta


KeyboardInterrupt: 